(COMPPARALELOGPUSSMC)=

# 5.5 Cómputo en paralelo usando GPUs en un sistema de memoria compartida (SMC)

```{admonition} Notas para contenedor de docker:

Comando de docker para ejecución de la nota de forma local:

nota: cambiar `<ruta a mi directorio>` por la ruta de directorio que se desea mapear a `/datos` dentro del contenedor de docker.

`docker run --rm -v <ruta a mi directorio>:/datos --name jupyterlab_optimizacion_2 -p 8888:8888 -p 8787:8787 -d palmoreck/jupyterlab_optimizacion_2:3.0.0`

password para jupyterlab: `qwerty`

Detener el contenedor de docker:

`docker stop jupyterlab_optimizacion_2`

Documentación de la imagen de docker `palmoreck/jupyterlab_optimizacion_2:3.0.0` en [liga](https://github.com/palmoreck/dockerfiles/tree/master/jupyterlab/optimizacion_2).

```

---

Nota generada a partir de [liga](https://www.dropbox.com/s/yjijtfuky3s5dfz/2.5.Compute_Unified_Device_Architecture.pdf?dl=0)

```{admonition} Al final de esta nota el y la lectora:
:class: tip

*

```

Se presentan códigos y sus ejecuciones en una máquina `p2.xlarge` de la nube de [AWS](https://aws.amazon.com/). Se utilizó la AMI:

```
opt2-aws-cuda-and-tools-08-05-2021
``` 

de la región `us-east-1` (Virginia) para reproducibilidad de resultados. Tal AMI se construyó a partir de una AMI `ubuntu 20.04 - ami-042e8287309f5df03` con el [script_cuda_and_tools.sh](https://github.com/palmoreck/scripts_for_useful_tools_installations/blob/main/AWS/ubuntu_20.04/optimizacion_2/script_cuda_and_tools.sh)

````{admonition} Comentario

Si se utiliza la *AMI* `opt2-aws-cuda-and-tools-08-05-2021` colocar en `User data` el siguiente *script*:

```bash

#!/bin/bash
##variables:
region=us-east-1 #make sure instance is in Virginia
name_instance=CUDA
USER=ubuntu
##System update
apt-get update -yq
##Tag instance
INSTANCE_ID=$(curl -s http://instance-data/latest/meta-data/instance-id)
PUBLIC_IP=$(curl -s http://instance-data/latest/meta-data/public-ipv4)
sudo -H -u $USER bash -c "/home/$USER/.local/bin/aws ec2 create-tags --resources $INSTANCE_ID --tag Key=Name,Value=$name_instance-$PUBLIC_IP --region=$region"
sudo -H -u $USER bash -c "cd / && /home/$USER/.local/bin/jupyter lab --ip=0.0.0.0 --no-browser --config=/home/$USER/.jupyter/jupyter_notebook_config.py &"

```

````

La máquina `p2.xlarge` tiene las siguientes características:

In [1]:
%%bash
lscpu

Architecture:                    x86_64
CPU op-mode(s):                  32-bit, 64-bit
Byte Order:                      Little Endian
Address sizes:                   46 bits physical, 48 bits virtual
CPU(s):                          4
On-line CPU(s) list:             0-3
Thread(s) per core:              2
Core(s) per socket:              2
Socket(s):                       1
NUMA node(s):                    1
Vendor ID:                       GenuineIntel
CPU family:                      6
Model:                           79
Model name:                      Intel(R) Xeon(R) CPU E5-2686 v4 @ 2.30GHz
Stepping:                        1
CPU MHz:                         2701.377
CPU max MHz:                     3000.0000
CPU min MHz:                     1200.0000
BogoMIPS:                        4600.15
Hypervisor vendor:               Xen
Virtualization type:             full
L1d cache:                       64 KiB
L1i cache:                       64 KiB
L2 cache:                        51

In [2]:
%%bash
sudo lshw -C memory

  *-firmware
       description: BIOS
       vendor: Xen
       physical id: 0
       version: 4.2.amazon
       date: 08/24/2006
       size: 96KiB
       capabilities: pci edd
  *-memory
       description: System Memory
       physical id: 1000
       size: 61GiB
       capabilities: ecc
       configuration: errordetection=multi-bit-ecc
     *-bank:0
          description: DIMM RAM
          physical id: 0
          slot: DIMM 0
          size: 16GiB
          width: 64 bits
     *-bank:1
          description: DIMM RAM
          physical id: 1
          slot: DIMM 1
          size: 16GiB
          width: 64 bits
     *-bank:2
          description: DIMM RAM
          physical id: 2
          slot: DIMM 2
          size: 16GiB
          width: 64 bits
     *-bank:3
          description: DIMM RAM
          physical id: 3
          slot: DIMM 3
          size: 13GiB
          width: 64 bits


In [3]:
%%bash
uname -ar #r for kernel, a for all

Linux ip-10-0-0-128 5.4.0-1045-aws #47-Ubuntu SMP Tue Apr 13 07:02:25 UTC 2021 x86_64 x86_64 x86_64 GNU/Linux


```{admonition} Observación
:class: tip

En la celda anterior se utilizó el comando de *magic* `%%bash`. Algunos comandos de *magic* los podemos utilizar también con `import`. Ver [ipython-magics](https://ipython.readthedocs.io/en/stable/interactive/magics.html#)

```

## *Compute Unified Device Architecture* (CUDA)

### Un poco de historia...

```{margin}

GPGPU es un término que se utilizó para referirse a la programación en unidades de procesamiento gráfico de forma general. Hoy en día se conoce simplemente como *GPU programming*. Ver [General-purpose computing on graphics processing units](https://en.wikipedia.org/wiki/General-purpose_computing_on_graphics_processing_units).

```

La industria de videojuegos impulsó el desarrollo de las tarjetas gráficas a una velocidad sin precedente a partir del año 1999 para incrementar el nivel de detalle visual en los juegos de video. Alrededor del 2003 se planteó la posibilidad de utilizar las unidades de procesamiento gráfico para procesamiento en paralelo relacionado con aplicaciones distintas al ambiente de gráficas. A partir del 2006 la empresa [NVIDIA](https://www.nvidia.com/en-us/about-nvidia/) introdujo CUDA, una plataforma GPGPU y un modelo de programación que facilita el procesamiento en paralelo en las GPU's.

Desde el 2006, las tarjetas gráficas muestran una brecha significativa con las unidades de procesamiento CPU's. Ver por ejemplo las gráficas que *NVIDIA* publica año tras año y que están relacionadas con el número de operaciones en punto flotante por segundo (FLOPS) y la transferencia de datos en la memoria RAM de la GPU: [gráficas cpu vs gpu en imágenes de google](https://www.google.com/search?q=plot+gflops+gpu+cpu+nvidia&tbm=isch&ved=2ahUKEwjKk7Le_bzwAhUUaKwKHX9-AP8Q2-cCegQIABAA&oq=plot+gflops+gpu+cpu+nvidia&gs_lcp=CgNpbWcQA1C_W1i_W2DhXGgAcAB4AIABX4gBX5IBATGYAQCgAQGqAQtnd3Mtd2l6LWltZ8ABAQ&sclient=img&ei=xAiYYMqhL5TQsQX__IH4Dw).

```{margin}

La GPU y la CPU están conectadas por una interconexión de nombre [PCI](https://en.wikipedia.org/wiki/Conventional_PCI).

```

Hoy en día se continúa el desarrollo de GPU's con mayor RAM, con mayor capacidad de cómputo y mejor conectividad con la CPU. Estos avances han permitido resolver problemas con mayor exactitud que los resueltos con las CPU's, por ejemplo en el terreno de *deep learning* en reconocimiento de imágenes. Ver [ImageNet Classification with Deep Convolutional Neural Networks](https://papers.nips.cc/paper/4824-imagenet-classification-with-deep-convolutional-neural-networks.pdf), [2012: A Breakthrough Year for Deep Learning](https://medium.com/limitlessai/2012-a-breakthrough-year-for-deep-learning-2a31a6796e73).



```{admonition} Observación
:class: tip

Para más avances ver [NVIDIA Turing Architecture In-Depth](https://devblogs.nvidia.com/nvidia-turing-architecture-in-depth/), [samsung-amd-rdna-gpu-2021](https://wccftech.com/samsung-amd-rdna-gpu-2021/), [playstation-5-specifications-revealed-but-design-is-still-a-mystery](https://www.theguardian.com/games/2020/mar/19/playstation-5-specifications-revealed-but-design-is-still-a-mystery), [xbox-series-x-tech](https://news.xbox.com/en-us/2020/03/16/xbox-series-x-tech/) y recientemente [IBM Supercomputer Summit Attacks Coronavirus…](https://www.ibm.com/blogs/nordic-msp/ibm-supercomputer-summit-attacks-coronavirus/). 

```

```{margin}

Recuérdese la [taxonomía de Flynn](https://en.wikipedia.org/wiki/Flynn%27s_taxonomy).

```

La arquitectura en la que podemos ubicar a las GPU's es en la de un sistema MIMD y SIMD. De hecho es [SIMT: Simple Instruction Multiple Thread](https://en.wikipedia.org/wiki/Single_instruction,_multiple_threads) en un modelo de sistema de memoria compartida pues "los *threads* en un *warp* leen la misma instrucción para ser ejecutada".

```{admonition} Definición

Un *warp* en el contexto de GPU *programming* es un conjunto de *threads*. Equivale a $32$ *threads*.

```

### ¿Diferencia con la CPU multicore?


<img src="https://dl.dropboxusercontent.com/s/k11qub01w4nvksi/CPU_multicore.png?dl=0" heigth="500" width="500">

**GPU**

<img src="https://dl.dropboxusercontent.com/s/lw9kia12qhwp95r/GPU.png?dl=0" heigth="500" width="500">

```{admonition} Observación
:class: tip

Obsérvese en el dibujo anterior la diferencia en tamaño del caché en la CPU y GPU. También la unidad de control es más pequeña en la GPU.


```

```{margin}

Una máquina *quad core* soporta cuatro threads en cada *core*.

```

A diferencia de una máquina *multicore* o multi CPU's con la habilidad de lanzar en un instante de tiempo unos cuantos *threads*, la GPU puede lanzar cientos o miles de threads en un instante siendo cada core *heavily multithreaded*. Sí hay restricciones en el número de threads que se pueden lanzar en un instante pues las tarjetas gráficas tienen diferentes características (modelo) y arquitecturas, pero la diferencia con la CPU es grande. Por ejemplo, la serie **GT 200** (2009) en un instante puede lanzar 30,720 threads con sus 240 *cores*. Ver [GeForce_200_series](https://en.wikipedia.org/wiki/GeForce_200_series), [List of NVIDIA GPU's](https://en.wikipedia.org/wiki/List_of_Nvidia_graphics_processing_units).

Ver [How Graphics Cards Work](https://computer.howstuffworks.com/graphics-card1.htm) y [How Microprocessors Work](https://computer.howstuffworks.com/microprocessor.htm) para más información.

### ¿Otras compañías producen tarjetas gráficas?

Sí, ver por ejemplo la lista de GPU's de [Advanced Micro Devices](https://en.wikipedia.org/wiki/List_of_AMD_graphics_processing_units).

### ¿Si tengo una tarjeta gráfica de AMD puedo correr un programa de CUDA?

No es posible pero algunas alternativas son:

* [OpenCl](https://www.khronos.org/opencl/)

* [OpenACC](https://www.openacc.org/about)

### ¿Si tengo una tarjeta gráfica de NVIDIA un poco antigua puedo correr un programa de CUDA?

Las GPU's producidas por NVIDIA desde 2006 son capaces de correr programas basados en ***CUDA C***. La cuestión sería revisar qué *compute capability* tiene tu tarjeta. Ver [Compute Capabilities](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities) para las características que tienen las tarjetas más actuales.

### ¿Qué es *CUDA C*?

Es una extensión al lenguaje *C* de programación en el que se utiliza una nueva sintaxis para procesamiento en la GPU. Contiene también una librería *runtime* que define funciones que se ejecutan desde el ***host*** por ejemplo para alojar y desalojar memoria en el ***device***, transferir datos entre la memoria *host* y la memoria *device* o manejar múltiples *devices*. La librería *runtime* está hecha encima de una API de *C* de bajo nivel llamada [NVIDIA CUDA Driver API](https://docs.nvidia.com/cuda/cuda-driver-api/index.html) la cual es accesible desde el código. Para información de la API de la librería runtime ver [NVIDIA CUDA Runtime API](https://docs.nvidia.com/cuda/cuda-runtime-api/index.html).

```{admonition} Comentario

La transferencia de datos entre la memoria del *host* a *device* o viceversa constituye un *bottleneck* fuerte.

```

### ¿A qué se refiere la terminología de *host* y *device*?

*Host* es la máquina *multicore* CPU y *device* es la GPU. Una máquina puede tener múltiples GPU's por lo que tendrá múltiples *devices*.

### Tengo una tarjeta NVIDIA CUDA *capable* ¿qué debo realizar primero?

Realizar instalaciones dependiendo de tu sistema operativo. Ver [instalación](https://github.com/palmoreck/programming-languages/tree/master/C/extensiones_a_C/CUDA/instalacion) donde además se encontrará información para instalación de [nvidia-docker](https://github.com/NVIDIA/nvidia-docker).

### Instalé lo necesario y al ejecutar en la terminal `nvcc -V` obtengo la versión... ¿cómo puedo probar mi instalación?

1)Obteniendo información del *NVIDIA driver* ejecutando en la terminal el comando `nvidia-smi`.

In [4]:
%%bash
nvidia-smi

Sun May  9 00:02:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 465.19.01    CUDA Version: 11.3     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA Tesla K80    On   | 00000000:00:1E.0 Off |                    0 |
| N/A   41C    P8    26W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
%%bash
nvidia-smi -a #a for all


==============NVSMI LOG==============

Timestamp                                 : Sun May  9 00:02:03 2021
Driver Version                            : 465.19.01
CUDA Version                              : 11.3

Attached GPUs                             : 1
GPU 00000000:00:1E.0
    Product Name                          : NVIDIA Tesla K80
    Product Brand                         : Tesla
    Display Mode                          : Disabled
    Display Active                        : Disabled
    Persistence Mode                      : Enabled
    MIG Mode
        Current                           : N/A
        Pending                           : N/A
    Accounting Mode                       : Disabled
    Accounting Mode Buffer Size           : 4000
    Driver Model
        Current                           : N/A
        Pending                           : N/A
    Serial Number                         : 0321417072356
    GPU UUID                              : GPU-c2ec0b4b-de80-2dfc-d3

Ver [results-for-the-nvidia-smi-command-in-a-terminal](https://askubuntu.com/questions/1220144/can-somebody-explain-the-results-for-the-nvidia-smi-command-in-a-terminal).

```{admonition} Comentario

Ejecutando `nvidia-smi -l 1` nos da información cada segundo.

```

2)Compilando y ejecutando el siguiente programa de *CUDA C*:

In [6]:
%%file hello_world.cu

#include<stdio.h>
__global__ void func(void){
    printf("Hello world! del bloque %d del thread %d\n", blockIdx.x, threadIdx.x);
}
int main(void){
    func<<<2,3>>>();
    cudaDeviceSynchronize();
    printf("Hello world! del cpu thread\n");
    return 0;
}


Writing hello_world.cu


```{admonition} Comentario

La sintaxis `<<<2,3>>>` refiere que serán lanzados 2 bloques de 3 *threads* cada uno.


```

Compilamos con `nvcc`.

In [7]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 hello_world.cu -o hello_world.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


```{admonition} Comentarios

* `nvcc` es un *wrapper* para el compilador de programas escritos en *C*.

* En ocasiones para tener funcionalidad de un determinado [compute capability](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities) se especifica la *flag* de `-arch=sm_11` en la línea de `nvcc`. En este caso se le indica al compilador que compile el programa para un *compute capability* de $1.1$. Ver [run a kernel using the larger grid size support offered](https://stackoverflow.com/questions/16954931/cuda-5-0-cudagetdeviceproperties-strange-grid-size-or-a-bug-in-my-code) por ejemplo.

* Para la versión 11 de CUDA se requiere explícitamente indicar la arquitectura y código para la compilación. Ver [cuda-11-kernel-doesnt-run](https://stackoverflow.com/questions/63675040/cuda-11-kernel-doesnt-run), [cuda-how-to-use-arch-and-code-and-sm-vs-compute](https://stackoverflow.com/questions/35656294/cuda-how-to-use-arch-and-code-and-sm-vs-compute/35657430#35657430), [cuda-compute-capability-requirements](https://stackoverflow.com/questions/28932864/cuda-compute-capability-requirements/28933055#28933055), [what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api](https://stackoverflow.com/questions/14038589/what-is-the-canonical-way-to-check-for-errors-using-the-cuda-runtime-api).

```

Ejecutamos.

In [8]:
%%bash
./hello_world.out

Hello world! del bloque 0 del thread 0
Hello world! del bloque 0 del thread 1
Hello world! del bloque 0 del thread 2
Hello world! del bloque 1 del thread 0
Hello world! del bloque 1 del thread 1
Hello world! del bloque 1 del thread 2
Hello world! del cpu thread


3)Haciendo un query a la GPU para ver qué características tiene (lo siguiente es posible ejecutar sólo si se instaló el [CUDA toolkit](https://developer.nvidia.com/cuda-toolkit)):

In [9]:
%%bash
cd /usr/local/cuda/samples/1_Utilities/deviceQuery/ && sudo make
/usr/local/cuda/samples/1_Utilities/deviceQuery/deviceQuery

/usr/local/cuda-11.3/bin/nvcc -ccbin g++   -m64      -gencode arch=compute_35,code=sm_35 -gencode arch=compute_37,code=sm_37 -gencode arch=compute_50,code=sm_50 -gencode arch=compute_52,code=sm_52 -gencode arch=compute_60,code=sm_60 -gencode arch=compute_61,code=sm_61 -gencode arch=compute_70,code=sm_70 -gencode arch=compute_75,code=sm_75 -gencode arch=compute_80,code=sm_80 -gencode arch=compute_86,code=sm_86 -gencode arch=compute_86,code=compute_86 -o deviceQuery deviceQuery.o 
mkdir -p ../../bin/x86_64/linux/release
cp deviceQuery ../../bin/x86_64/linux/release
/usr/local/cuda/samples/1_Utilities/deviceQuery/deviceQuery Starting...

 CUDA Device Query (Runtime API) version (CUDART static linking)

Detected 1 CUDA Capable device(s)

Device 0: "NVIDIA Tesla K80"
  CUDA Driver Version / Runtime Version          11.3 / 11.3
  CUDA Capability Major/Minor version number:    3.7
  Total amount of global memory:                 11441 MBytes (11996954624 bytes)
  (013) Multiprocessors, (192) 

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


### ¿Por qué usar CUDA y *CUDA-C* o más general cómputo en la GPU?

* NVIDIA como se mencionó al inicio de la nota fue de las primeras compañías en utilizar la GPU para tareas no relacionadas con el área de gráficos, ha colaborado en el avance del conocimiento de las GPU's y desarrollo de algoritmos y tarjetas gráficas. Otra compañía es [Khronos_Group](https://en.wikipedia.org/wiki/Khronos_Group) por ejemplo, quien actualmente desarrolla [OpenCl](https://www.khronos.org/opencl/).

```{margin}

*Deep learning* se ha utilizado para resolver problemas en *machine learning* típicos. Ejemplos de esto son la clasificación de imágenes, de sonidos o análisis de textos. Ver por ejemplo [Practical text analysis using deep learning](https://medium.com/@michael.fire/practical-text-analysis-using-deep-learning-5fb0744efdf9).

```

* El cómputo en la GPU constituye hoy en día una alternativa fuerte a la implementación de modelos de *machine learning* ampliamente utilizada por la comunidad científica, también para cómputo matricial y *deep learning*.

* Sí hay publicaciones científicas para la implementación de *deep learning* en las CPU's, ver por ejemplo el *paper* reciente de [SLIDE](https://www.cs.rice.edu/~as143/Papers/SLIDE_MLSys.pdf) cuyo repo de *github* es [HashingDeepLearning](https://github.com/keroro824/HashingDeepLearning). Tal *paper* plantea una discusión a realizar con la frase:

*...change in the state-of-the-art algorithms can render specialized hardware less effective in the future*. 

Ver por ejemplo [Tensor Cores](https://developer.nvidia.com/tensor-cores), [NVIDIA TENSOR CORES, The Next Generation of Deep Learning](https://www.nvidia.com/en-us/data-center/tensorcore/), [The most powerful computers on the planet: SUMMIT](https://www.ibm.com/thought-leadership/summit-supercomputer/) como ejemplos de hardware especializado para aprendizaje con *Tensorflow*.

```{admonition} Observación
:class: tip

*Summit powered by 9,126 IBM Power9 CPUs and over 27,000 NVIDIA V100 Tensor Core GPUS, is able to do 200 quadrillion calculations per second...* [IBM Supercomputer Summit Attacks Coronavirus…](https://www.ibm.com/blogs/nordic-msp/ibm-supercomputer-summit-attacks-coronavirus/).

```

Sin embargo, por falta de implementaciones algorítmicas en la *CPU* se han adoptado implementaciones de *deep learning* utilizando GPU's:


*...However, for the case of DL, this investment is justified due to the lack of significant progressin the algorithmic alternatives for years.*

```{admonition} Comentario

Revisar también las entradas [An algorithm could make CPUs a cheap way to train AI](https://www.engadget.com/2020/03/03/rice-university-slide-cpu-gpu-machine-learning/) y [Deep learning rethink overcomes major obstacle in AI industry](https://www.sciencedaily.com/releases/2020/03/200305135041.htm). 

```

## [CUDA-C](https://docs.nvidia.com/cuda/cuda-c-programming-guide/)

Consiste en extensiones al lenguaje C y en una *runtime library*.

### *Kernel*

* En *CUDA C* se define una función que se ejecuta en el ***device*** y que se le nombra ***kernel***. El *kernel* inicia con la sintaxis:

```C
__global__ void mifun(int param){
...
}

```

* Siempre es tipo `void` (no hay `return`).

* El llamado al *kernel* se realiza desde el ***host*** y con una sintaxis en la que se define el número de *threads*, nombrados ***CUDA threads*** (que son distintos a los *CPU threads*), y bloques, nombrados ***CUDA blocks***, que serán utilizados para la ejecución del *kernel*. La sintaxis que se utiliza es `<<< >>>` y en la primera entrada se coloca el número de *CUDA blocks* y en la segunda entrada el número de *CUDA threads*. Por ejemplo para lanzar N bloques de 5 *threads*.


```C
__global__ void mifun(int param){
...
}

int main(){
    int par=0;
    mifun<<<N,5>>> (par); 
}

```

### Ejemplo

`hello_world_simple.cu`

In [10]:
%%file hello_world_simple.cu
#include<stdio.h>
__global__ void func(void){
}
int main(void){
    func<<<1,1>>>();
    printf("Hello world!\n");
return 0;
}

Writing hello_world_simple.cu


Compilación:

In [11]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall hello_world_simple.cu -o hello_world_simple.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


Ejecución:

In [12]:
%%bash
./hello_world_simple.out

Hello world!


````{admonition} Comentarios

* La función `main` se ejecuta en la CPU.

* `func` es un *kernel* y es ejecutada por los *CUDA threads* en el *device*. Obsérvese que tal función inicia con la sintaxis `__global__`. En este caso el *CUDA thread* que fue lanzado no realiza ninguna acción pues el cuerpo del kernel está vacío.

* El *kernel* sólo puede tener un `return` tipo *void*: `__global__ void func` por lo que el *kernel* debe regresar sus resultados a través de sus argumentos.
 
* La extensión del archivo debe ser `.cu` aunque esto puede modificarse al compilar con `nvcc`: 

```bash

nvcc -x cu hello_world.c -o hello_world.out

```

````

### ¿Bloques de threads? 

Los *CUDA threads* son divididos en *CUDA blocks* y éstos se encuentran en un *grid*. En el lanzamiento del *kernel* se debe especificar al hardware cuántos *CUDA blocks* tendrá nuestro *grid* y cuántos *CUDA threads* estarán en cada bloque. 

### Ejemplo

```{margin}

`func<<<2,3>>>();` representa 2 bloques de 3 *threads* cada uno.

```

In [13]:
%%file hello_world_2.cu
#include<stdio.h>
__global__ void func(void){
    printf("Hello world! del bloque %d del thread %d\n", blockIdx.x, threadIdx.x);
}
int main(void){
    func<<<2,3>>>();
    cudaDeviceSynchronize();
    //printf("Hello world! del cpu thread\n");
    return 0;
}


Writing hello_world_2.cu


In [14]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall hello_world_2.cu -o hello_world_2.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [15]:
%%bash
./hello_world_2.out

Hello world! del bloque 0 del thread 0
Hello world! del bloque 0 del thread 1
Hello world! del bloque 0 del thread 2
Hello world! del bloque 1 del thread 0
Hello world! del bloque 1 del thread 1
Hello world! del bloque 1 del thread 2


**En lo que continúa de la nota el nombre *thread* hará referencia a *CUDA thread* y el nombre bloque a *CUDA block*.**

```{admonition} Comentarios

* El llamado a la ejecución del *kernel* se realizó en el *host* y se lanzaron $2$ bloques cada uno con $3$ *threads*.

* Se utiliza la función [cudaDeviceSynchronize](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__DEVICE.html#group__CUDART__DEVICE_1g10e20b05a95f638a4071a655503df25d) para que el *cpu-thread* espere la finalización de la ejecución del *kernel*.

* En el ejemplo anterior, las variables `blockIdx` y `threadIdx` hacen referencia a los **id**'s que tienen los bloques y los *threads*. El *id* del bloque dentro del *grid* y el *id* del thread dentro del bloque. La parte `.x` de las variables: `blockIdx.x` y `threadIdx.x` refieren a la **primera coordenada** del bloque en el *grid* y a la **primera coordenada** del *thread* en en el bloque. 

* La elección del número de bloques en un *grid* o el número de *threads* en un bloque no corresponde a alguna disposición del *hardware*. Esto es, si se lanza un *kernel* con `<<< 1, 3 >>>` no implica que la GPU tenga en su *hardware* un bloque o 3 *threads*. Asimismo, las coordenadas que se obtienen vía `blockIdx` o `threadIdx` son meras abstracciones, no corresponden a algún ordenamiento en el hardware de la GPU.

* Todos los *threads* de un bloque  ejecutan el *kernel* por lo que se tienen tantas copias del kernel como número de bloques sean lanzados. Esto es una muestra la GPU sigue el modelo  *Single Instruction Multiple Threads [(SIMT)](https://en.wikipedia.org/wiki/Single_instruction,_multiple_threads)*.

```

### ¿Grid's y bloques 3-dimensionales?

En el *device* podemos definir el *grid* de bloques y el bloque de *threads* utilizando el tipo de dato `dim3` el cual también es parte de *CUDA C*.

### Ejemplo

```{margin}

`dim3 dimGrid(1,2,1);` representa 2 bloques en el *grid*.

`dim3 dimBlock(1,1,3);` representa 3 *threads* por bloque.

```

In [1]:
%%file hello_world_3.cu
#include<stdio.h>
__global__ void func(void){
    printf("Hello world! del bloque %d del thread %d\n", blockIdx.y, threadIdx.z);
}
int main(void){
    dim3 dimGrid(1,2,1);
    dim3 dimBlock(1,1,3);
    func<<<dimGrid,dimBlock>>>(); 
    cudaDeviceSynchronize();
    printf("Hello world! del cpu thread\n");
    return 0;
}

Writing hello_world_3.cu


In [17]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall hello_world_3.cu -o hello_world_3.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [18]:
%%bash
./hello_world_3.out

Hello world! del bloque 0 del thread 0
Hello world! del bloque 0 del thread 1
Hello world! del bloque 0 del thread 2
Hello world! del bloque 1 del thread 0
Hello world! del bloque 1 del thread 1
Hello world! del bloque 1 del thread 2
Hello world! del cpu thread


### Ejemplo

```{margin}

`dim3 dimGrid(1,1,1);` representa 1 bloque en el *grid*.

`dim3 dimBlock(1,3,1);` representa 3 *threads* por bloque.

```

In [22]:
%%file thread_idxs.cu
#include<stdio.h>
__global__ void func(void){
    if(threadIdx.x==0 && threadIdx.y==0 && threadIdx.z==0){
        printf("blockIdx.x:%d\n",blockIdx.x);
    }
    printf("thread idx.x:%d\n",threadIdx.x);
    printf("thread idx.y:%d\n",threadIdx.y);
    printf("thread idx.z:%d\n",threadIdx.z);
}
int main(void){
    dim3 dimGrid(1,1,1);
    dim3 dimBlock(1,3,1);
    func<<<dimGrid,dimBlock>>>(); 
    cudaDeviceSynchronize();
    return 0;
}

Writing thread_idxs.cu


In [23]:
%%bash 
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 thread_idxs.cu -o thread_idxs.out 

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [24]:
%%bash
./thread_idxs.out

blockIdx.x:0
thread idx.x:0
thread idx.x:0
thread idx.x:0
thread idx.y:0
thread idx.y:1
thread idx.y:2
thread idx.z:0
thread idx.z:0
thread idx.z:0


### Ejemplo

```{margin}

`dim3 dimGrid(1,2,2);` representa 4 bloques en el *grid*.

`dim3 dimBlock(1,1,1);` representa 1 *thread* por bloque.

```

In [25]:
%%file block_idxs.cu
#include<stdio.h>
__global__ void func(void){
    printf("blockIdx.x:%d\n",blockIdx.x);
    printf("blockIdx.y:%d\n",blockIdx.y);
    printf("blockIdx.z:%d\n",blockIdx.z);

}
int main(void){
    dim3 dimGrid(1,2,2); 
    dim3 dimBlock(1,1,1); 
    func<<<dimGrid,dimBlock>>>(); 
    cudaDeviceSynchronize();
    return 0;
}

Writing block_idxs.cu


In [26]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall block_idxs.cu -o block_idxs.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [27]:
%%bash
./block_idxs.out

blockIdx.x:0
blockIdx.x:0
blockIdx.x:0
blockIdx.x:0
blockIdx.y:1
blockIdx.y:0
blockIdx.y:1
blockIdx.y:0
blockIdx.z:1
blockIdx.z:0
blockIdx.z:0
blockIdx.z:1


### Ejemplo

Podemos usar la variable `blockDim` para cada coordenada `x, y` o `z` y obtener la dimensión de los bloques.

```{margin}

`dim3 dimGrid(2,2,2);` representa 8 bloques en el *grid*.

`dim3 dimBlock(3,1,2);` representa 6 *threads* por bloque.

```

In [30]:
%%file block_dims.cu
#include<stdio.h>
__global__ void func(void){
    if(threadIdx.x==0 && threadIdx.y==0 && threadIdx.z==0 && blockIdx.z==1){
    printf("blockDim.x:%d\n",blockDim.x);
    printf("blockDim.y:%d\n",blockDim.y);
    printf("blockDim.z:%d\n",blockDim.z);
    }

}
int main(void){
    dim3 dimGrid(2,2,2);
    dim3 dimBlock(3,1,2);
    func<<<dimGrid,dimBlock>>>(); 
    cudaDeviceSynchronize();
    return 0;
}

Writing block_dims.cu


In [31]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall block_dims.cu -o block_dims.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [32]:
%%bash
./block_dims.out

blockDim.x:3
blockDim.x:3
blockDim.x:3
blockDim.x:3
blockDim.y:1
blockDim.y:1
blockDim.y:1
blockDim.y:1
blockDim.z:2
blockDim.z:2
blockDim.z:2
blockDim.z:2


### Alojamiento de memoria en el *device*

Para alojar memoria en el *device* se utiliza el llamado a [cudaMalloc](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1g37d37965bfb4803b6d4e59ff26856356) y para transferir datos del *host* al *device* o viceversa se llama a lafunción [cudaMemcpy](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1gc263dbe6574220cc776b45438fc351e8) con respectivos parámetros como `cudaMemcpyHostToDevice` o `cudaMemcpyDeviceToHost`. 

Para desalojar memoria del *device* se utiliza el llamado a [cudaFree](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__MEMORY.html#group__CUDART__MEMORY_1ga042655cbbf3408f01061652a075e094).

### Ejemplo

**N bloques de 1 thread**

In [33]:
%%file vector_sum.cu
#include<stdio.h>
#define N 10
__global__ void vect_sum(int *a, int *b, int *c){
    int block_id_x = blockIdx.x;
    if(block_id_x<N) //we assume N is less than maximum number of blocks
                     //that can be launched
        c[block_id_x] = a[block_id_x]+b[block_id_x];
}
int main(void){
    int a[N], b[N],c[N];
    int *device_a, *device_b, *device_c;
    int i;
    dim3 dimGrid(N,1,1);
    dim3 dimBlock(1,1,1);
    //allocation in device
    cudaMalloc((void **)&device_a, sizeof(int)*N); 
    cudaMalloc((void **)&device_b, sizeof(int)*N);
    cudaMalloc((void **)&device_c, sizeof(int)*N);
    //dummy data
    for(i=0;i<N;i++){
        a[i]=i;
        b[i]=i*i;
    }
    //making copies of a, b arrays to GPU
    cudaMemcpy(device_a,a,N*sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(device_b,b,N*sizeof(int), cudaMemcpyHostToDevice);
    //mandamos a llamar a suma_vect:
    vect_sum<<<dimGrid,dimBlock>>>(device_a,device_b,device_c);
    cudaDeviceSynchronize();
    //copy result to c array
    cudaMemcpy(c,device_c,N*sizeof(int),cudaMemcpyDeviceToHost);
    for(i=0;i<N;i++)
        printf("%d+%d = %d\n",a[i],b[i],c[i]);
    cudaFree(device_a);
    cudaFree(device_b);
    cudaFree(device_c);
    return 0;
}

Writing vector_sum.cu


```{admonition} Comentarios

* `dim3 dimGrid(N,1,1);` representa N bloques en el *grid*.

* `dim3 dimBlock(1,1,1);` representa 1 *thread* por bloque.

* `<<<dimGrid,dimBlock>>>` N bloques de 1 *thread*.

```

In [34]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall vector_sum.cu -o vector_sum.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [35]:
%%bash
./vector_sum.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


````{admonition} Comentarios

* Los *statements*:

```C
    int *device_a, *device_b, *device_c;
```

en sintaxis de *C* se definen apuntadores que refieren a una dirección de memoria. En el contexto de la *GPU programming* estos apuntadores no apuntan a una dirección de memoria en el *device*. Aunque NVIDIA añadió el *feature* de [Unified Memory](https://devblogs.nvidia.com/unified-memory-cuda-beginners/) (un espacio de memoria accesible para el *host* y el *device*) aquí no se está usando tal *feature*. Más bien se están utilizando los apuntadores anteriores para apuntar a un [struct](https://en.wikipedia.org/wiki/Struct_(C_programming_language)) de *C* en el que uno de sus tipos de datos es una dirección de memoria en el *device*.

* El uso de `(void **)` es por la definición de la función `cudaMalloc`.

* En el programa anterior se coloca en comentario que se asume que $N$ el número de datos en el arreglo es menor al número de bloques que es posible lanzar. Esto como veremos más adelante es importante considerar pues aunque en un *device* se pueden lanzar muchos bloques y muchos threads, se tienen límites en el número de éstos que es posible lanzar.


````

### ¿Perfilamiento en CUDA?

Al instalar el *CUDA toolkit* en sus máquinas o bien si utilizan el contenedor de docker (descrito al inicio de la nota) se instala la línea de comando [nvprof](https://docs.nvidia.com/cuda/profiler-users-guide/index.html) para perfilamiento. Se puede ejecutar con:

In [36]:
%%bash
source ~/.profile
nvprof --normalized-time-unit s ./vector_sum.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


==6052== NVPROF is profiling process 6052, command: ./vector_sum.out
==6052== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
==6052== Profiling application: ./vector_sum.out
==6052== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:    44.81  5.25e-06         1  5.25e-06  5.25e-06  5.25e-06  vect_sum(int*, int*, int*)
                    30.87  3.62e-06         2  1.81e-06  1.54e-06  2.08e-06  [CUDA memcpy HtoD]
                    24.32  2.85e-06         1  2.85e-06  2.85e-06  2.85e-06  [CUDA memcpy DtoH]
      API calls:    99.60  0.269994         3  0.089998  3.85e-06  0.269984  cudaMalloc
                     0.19  5.14e-04         1  5.14e-04  5.14e-04  5.14e-04  cuDeviceTotalMem
                     0.10  2.74e-04       101  2.71e-06  7.41e-07  9.10e-05  cuDeviceGetAttribute
                     0.06  1.56e

**Comentarios:**

* Las unidades en las que se reporta son s: second, ms: millisecond, us: microsecond, ns: nanosecond.

* En la documentación de NVIDIA se menciona que `nvprof` será reemplazada próximamente por [NVIDIA Nsight Compute](https://developer.nvidia.com/nsight-compute) y [NVIDIA Nsight Systems](https://developer.nvidia.com/nsight-systems).


En el ejemplo anterior se lanzaron $N$ bloques con $1$ *thread* cada uno y a continuación se lanza $1$ bloque con $N$ *threads*.

In [37]:
%%file vector_sum_2.cu
#include<stdio.h>
#define N 10
__global__ void vect_sum(int *a, int *b, int *c){
    int thread_id_x = threadIdx.x;
    if(thread_id_x<N) 
        c[thread_id_x] = a[thread_id_x]+b[thread_id_x];
}
int main(void){
    int *device_a, *device_b, *device_c;
    int i;
    dim3 dimGrid(1,1,1);
    dim3 dimBlock(N,1,1);
    //alojando en device con Unified Memory
    cudaMallocManaged(&device_a, sizeof(int)*N);
    cudaMallocManaged(&device_b, sizeof(int)*N);
    cudaMallocManaged(&device_c, sizeof(int)*N);
    //llenando los arreglos:
    for(i=0;i<N;i++){
        device_a[i]=i;
        device_b[i]=i*i;
    }
    vect_sum<<<dimGrid,dimBlock>>>(device_a,device_b,device_c); 
    cudaDeviceSynchronize();
    for(i=0;i<N;i++)
        printf("%d+%d = %d\n",device_a[i],device_b[i],device_c[i]);
    cudaFree(device_a);
    cudaFree(device_b);
    cudaFree(device_c);
    return 0;
}

Writing vector_sum_2.cu


```{admonition} Comentarios

* `dim3 dimGrid(1,1,1);` representa 1 bloque en el *grid*.

* `dim3 dimBlock(N,1,1);` representa N *threads* por bloque.

* `<<<dimGrid,dimBlock>>>` 1 bloque con N *threads*.

```

In [38]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall vector_sum_2.cu -o vector_sum_2.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [39]:
%%bash
source ~/.profile
nvprof --normalized-time-unit s ./vector_sum_2.out

0+0 = 0
1+1 = 2
2+4 = 6
3+9 = 12
4+16 = 20
5+25 = 30
6+36 = 42
7+49 = 56
8+64 = 72
9+81 = 90


==6101== NVPROF is profiling process 6101, command: ./vector_sum_2.out
==6101== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
==6101== Profiling application: ./vector_sum_2.out
==6101== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:   100.00  5.79e-06         1  5.79e-06  5.79e-06  5.79e-06  vect_sum(int*, int*, int*)
      API calls:    99.58  0.303429         3  0.101143  7.14e-06  0.303394  cudaMallocManaged
                     0.17  5.21e-04         1  5.21e-04  5.21e-04  5.21e-04  cuDeviceTotalMem
                     0.09  2.86e-04       101  2.83e-06  7.94e-07  9.55e-05  cuDeviceGetAttribute
                     0.08  2.45e-04         1  2.45e-04  2.45e-04  2.45e-04  cudaLaunchKernel
                     0.05  1.62e-04         3  5.41e-05  9.79e-06  1.26e-04  cudaFree
                     0.01  3.50e-

**Comentarios:**

* El programa anterior utiliza la [Unified Memory](https://devblogs.nvidia.com/unified-memory-cuda-beginners/) con la función [cudaMallocManaged](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__HIGHLEVEL.html#group__CUDART__HIGHLEVEL_1gcf6b9b1019e73c5bc2b39b39fe90816e). La *Unified Memory* es un *feature* que se añadió a CUDA desde las arquitecturas de **Kepler** y **Maxwell** pero que ha ido mejorando (por ejemplo añadiendo [page faulting](https://en.wikipedia.org/wiki/Page_fault) and [migration](https://www.kernel.org/doc/html/latest/vm/page_migration.html)) en las arquitecturas siguientes a la de *Kepler*: la arquitectura Pascal y Volta. Por esto en el *output* anterior de *nvprof* aparece una sección de *page fault*. 

* Obsérvese que en el programa anterior se comenta que se asume que $N$ el número de datos en el arreglo es menor al número de *threads* que es posible lanzar. Esto como veremos más adelante es importante considerar pues aunque en el *device* se pueden lanzar muchos bloques y muchos threads, se tienen límites en el número de éstos que es posible lanzar.


### ¿Tenemos que inicializar los datos en la CPU y copiarlos hacia la GPU?

En realidad no tenemos que realizarlo para el ejemplo de `suma_vectorial.cu` anterior. Por ejemplo:

In [44]:
%%file vector_sum_3.cu
#include<stdio.h>
#define N 10
__global__ void fill_arrays(int *a, int *b){
    int thread_id_x = threadIdx.x;
    a[thread_id_x]=thread_id_x;
    b[thread_id_x]=thread_id_x*thread_id_x;
}

__global__ void vect_sum(int *a, int *b, int *c){
    int thread_id_x = threadIdx.x;
    if(thread_id_x<N)
        c[thread_id_x] = a[thread_id_x]+b[thread_id_x];
}

int main(void){
    int *device_a, *device_b, *device_c;
    int i;
    dim3 dimGrid(1,1,1);
    dim3 dimBlock(N,1,1);
    //allocating using Unified Memory in device
    cudaMallocManaged(&device_a, sizeof(int)*N);
    cudaMallocManaged(&device_b, sizeof(int)*N);
    cudaMallocManaged(&device_c, sizeof(int)*N);
    fill_arrays<<<dimGrid,dimBlock>>>(device_a,device_b);
    cudaDeviceSynchronize();
    vect_sum<<<dimGrid,dimBlock>>>(device_a,device_b,device_c);
    cudaDeviceSynchronize();
    for(i=0;i<N;i++)
        printf("%d+%d = %d\n",device_a[i],device_b[i],device_c[i]);
    cudaFree(device_a);
    cudaFree(device_b);
    cudaFree(device_c);
    return 0;
}

Writing vector_sum_3.cu


In [45]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall vector_sum_3.cu -o vector_sum_3.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [ ]:
%%bash
source ~/.profile
nvprof --normalized-time-unit s ./vector_sum_3.out

## Arquitectura de una GPU y límites en número de threads y bloques que podemos lanzar en el kernel

Un *device* está compuesto por arreglos de **streaming multiprocessors SM's** (también denotados como MP's) y en cada *SM* encontramos un número (determinado por la arquitectura del device) de **streaming processors SP's** que comparten el caché y unidades de control (que están dentro de cada SM):

<img src="https://dl.dropboxusercontent.com/s/oxx55upoayfmliw/SMS_CUDA.png?dl=0" heigth="700" width="700">


En el dibujo anterior se muestran las SM's en color rojo y los SP's en morado. Hay dos SM's por cada bloque anaranjado y ocho SP's por cada SM. Así, una GPU es una máquina multicore. Aunque cada SM ejecuta las instrucciones de forma independiente a otra SM, comparten la **memoria global**.

Los bloques de threads son **asignados a cada SM por el CUDA runtime system**, puede asignar más de un bloque a una SM pero hay un límite de bloques que pueden ser asignados a cada SM. Ver [maximum number of blocks per multiprocessor](https://stackoverflow.com/questions/22520209/programmatically-retrieve-maximum-number-of-blocks-per-multiprocessor).

**Comentarios:** 

* Por ejemplo para el modelo *GT200* el máximo número de bloques que podían asignarse a cada SM eran de $8$ bloques. Tal modelo tenía $30$ SM's lo que resultaban en $240$ bloques que en un instante podían asignarse al device para su ejecución simultánea (asignándose en cualquier orden en alguna SM disponible). Por supuesto que un grid podía contener más de $240$ bloques en este modelo y en este caso el *CUDA runtime system* lleva una lista de bloques que va asignando a cada SM y conforme cada SM terminan la ejecución, nuevos bloques son asignados a tales SM que finalizaron. Para visualizar esta situación, considérese una simplificación de lo anterior en donde se tiene un *device* con $2$ SM's y con un kernel se han lanzado $6$ bloques. El *CUDA runtime system* ha asignado $3$ bloques a cada SM, entonces se tiene un dibujo como el siguiente:


<img src="https://dl.dropboxusercontent.com/s/p0nu72ofmdjtck8/kernel_launch_example.png?dl=0" heigth="600" width="600">


* Los bloques asignados a una SM comparten recursos (por ejemplo memoria) y su ejecución es independiente entre ellos, no es posible sincronizar al "bloque 1" con el "bloque 0". También no es posible sincronizar a los *threads* de diferentes SM's pero sí es posible sincronizar a los *threads* dentro de un mismo bloque.

### ¿Qué otros límites puedo encontrar en mi(s) device(s) de mi sistema?

Para responder lo anterior se puede utilizar el siguiente programa que está basado en [liga](https://devblogs.nvidia.com/how-query-device-properties-and-handle-errors-cuda-cc/) y [cudaDeviceProp Struct Reference](https://docs.nvidia.com/cuda/cuda-runtime-api/structcudaDeviceProp.html):

In [1]:
%%file device_properties.cu

#include<stdio.h>

int main(void){
    cudaDeviceProp properties;
    int count;
    int i;
    cudaGetDeviceCount(&count);
    for(i=0;i<count;i++){
        printf("----------------------\n");
        cudaGetDeviceProperties(&properties, i);
        printf("----device %d ----\n",i); 
        printf("Device Name: %s\n", properties.name);
        printf("Compute capability: %d.%d\n", properties.major, properties.minor);
        printf("Clock rate: %d\n", properties.clockRate);
        printf("Unified memory: %d\n", properties.unifiedAddressing);
        printf(" ---Memory Information for device %d (results on bytes)---\n", i);
        printf("Total global mem: %ld\n", properties.totalGlobalMem); 
        printf("Total constant Mem: %ld\n", properties.totalConstMem);
        printf("Shared memory per thread block: %ld\n", properties.sharedMemPerBlock);
        printf("Shared memory per SM: %ld\n",properties.sharedMemPerMultiprocessor );
        printf(" ---MP Information for device %d ---\n", i);
        printf("SM count: %d\n", properties.multiProcessorCount);
        printf("Threads in warp: %d\n", properties.warpSize);
        printf("Max threads per SM: %d\n", properties.maxThreadsPerMultiProcessor);
        printf("Max warps per SM: %d\n",properties.maxThreadsPerMultiProcessor/properties.warpSize);
        printf("Max threads per block: %d\n", properties.maxThreadsPerBlock);
        printf("Max thread dimensions: (%d, %d, %d)\n", properties.maxThreadsDim[0], properties.maxThreadsDim[1], properties.maxThreadsDim[2]);
        printf("Max grid dimensions: (%d, %d, %d)\n", properties.maxGridSize[0], properties.maxGridSize[1], properties.maxGridSize[2]); 
    }
    return 0;
    
}

Writing device_properties.cu


In [2]:
%%bash
source ~/.profile
nvcc --compiler-options -Wall device_properties.cu -o device_properties.out

In [3]:
%%bash
./device_properties.out

----------------------
----device 0 ----
Device Name: NVIDIA Tesla K80
Compute capability: 3.7
Clock rate: 823500
Unified memory: 1
 ---Memory Information for device 0 (results on bytes)---
Total global mem: 11996954624
Total constant Mem: 65536
Shared memory per thread block: 49152
Shared memory per SM: 114688
 ---MP Information for device 0 ---
SM count: 13
Threads in warp: 32
Max threads per SM: 2048
Max warps per SM: 64
Max threads per block: 1024
Max thread dimensions: (1024, 1024, 64)
Max grid dimensions: (2147483647, 65535, 65535)


**Comentarios:**

* También en la documentación oficial de NVIDIA dentro de [compute-capabilities](https://docs.nvidia.com/cuda/cuda-c-programming-guide/index.html#compute-capabilities) se pueden revisar los valores anteriores y muchos más.

* Obsérvese del *output* anterior que el sistema tiene dos *devices* con las mismas capacidades.

* En un *device* encontramos diferentes tipos de memoria: global, constante, *shared* y *texture*. En esta nota únicamente trabajamos con la memoria global. Tenemos funciones en CUDA para poder comunicar/coordinar a los *threads* en un bloque por medio de la *shared memory*. Ver por ejemplo [Using Shared Memory in CUDA C/C++](https://devblogs.nvidia.com/using-shared-memory-cuda-cc/) para un pequeño *post* del $2013$ sobre *shared memory*.

* Los bloques de threads que son asignados a una SM son divididos en **warps** que es la unidad de **thread scheduling\*** que tiene el *CUDA run time system*. El *output* anterior indica que son divisiones de $32$ threads: un warp consta de $32$ threads.

\* El *thread scheduling* se puede pensar a la funcionalidad del hardware para seleccionar una instrucción del programa y asginar su ejecución por los *threads* en un *warp* ([SIMT](https://en.wikipedia.org/wiki/Single_instruction,_multiple_threads)). Otro ejemplo es tener una instrucción que indica que se debe realizar lectura o escritura, entonces el hardware del *device* utiliza un *warp* de threads para tal operación mientras selecciona un *warp* de threads distinto para seleccionar otra instrucción diferente a la de I/O.

* El número máximo de threads que pueden iniciarse de forma simultánea o en un instante por SM es de $2048$ o bien $2048/32 = 64$ warps.

* El *output* anterior muestra los límites para número de bloques en las tres dimensiones de un grid y el número de threads en las tres dimensiones en un bloque.

* Un bloque puede tener como máximo $1024$ threads en cualquier configuración: por ejemplo $(1024,1,1), (32,1,32), (4,4,64)$.

* Por los puntos anteriores si lanzamos bloques de $1024$ threads entonces sólo $2$ bloques pueden residir en una SM en un instante. Con esta configuración alcanzaríamos $1024/32=32$ warps por cada bloque y como lanzamos $2$ bloques alcanzaríamos $64$ warps (que es el máximo de warps por SM que podemos tener en un instante). Otra configuración para alcanzar el máximo número de warps en un instante, es considerar $4$ bloques de $512$ threads pues tendríamos $512/32=16$ warps por bloque y en total serían $16*4$ (warps $\times$ bloques) $=64$ warps. Entre los datos que hay que elegir en los programas de CUDA C se tienen las configuraciones en el número de threads y el número de bloques a lanzar. La idea es alcanzar o rebasar el máximo número de warps en cada SM que soporta nuestro device en un instante.

* Por ejemplo para el dibujo en el que se asumió que el *CUDA runtime system* había asignado $3$ bloques a cada SM, se tendría una división de cada bloque en un *warp* de $32$ threads como sigue:


<img src="https://dl.dropboxusercontent.com/s/yngq4r66i2nk5mg/warp_division.png?dl=0" heigth="600" width="600">



### Grid Configuration Choices?

Los programas de CUDA C tienen la opción de elegir el número de *threads* y de *bloques* a ser lanzados. En la referencia "Parallel Computing for Data Science. With Examples in R, C++ and CUDA" de N. Matloff se enlistan algunas consideraciones para elegir tales parámetros:

* Given that scheduling is done on a warp basis, block size should be a multiple of the warp size (32).

* One wants to utilize all the SMs. If one sets the block size too large, not all will be used, as a block cannot be split across SM's.

* ..., barrier synchronization can be done effectively only at the block level. The larger the block, the more the barrier delay, so one might want smaller blocks.

* On the other hand, if one is using shared memory, this can only be done at the block level, and efficient use may indicate using a larger block.

* Two threads doing unrelated work, or the same work but with many if/elses, would cause a lot of thread divergence if they were in the same block. In some cases, it may be known in advance which threads will do the "ifs" and which will do the "elses", in which case they should be placed in different blocks if possible.

* A commonly-cited rule of thumb is to have between $128$ and $256$ *threads* per block.

### Regla compuesta del rectángulo

En el uso de CUDA se recomienda que:

* Users escriban código de CUDA C simple.

* Utilicen las librerías ya hechas por NVIDIA o terceros para mantener simplicidad y eficiencia en el código.

Lo anterior para disminuir el tiempo y la cantidad de código que *users* tengan que hacer (o rehacer) y puesto que dominar la programación de CUDA C requiere una buena inversión de tiempo.

Así, tenemos a [Thrust](https://docs.nvidia.com/cuda/thrust/index.html) una *template library* basada en la [Standard Template Library (STL)](https://en.wikipedia.org/wiki/Standard_Template_Library) de C++ construída por NVIDIA que de acuerdo a su documentación: 

"Thrust provides a rich collection of data parallel primitives such as scan, sort, and reduce, which can be composed together to implement complex algorithms with concise, readable source code. By describing your computation in terms of these high-level abstractions you provide Thrust with the freedom to select the most efficient implementation automatically. As a result, Thrust can be utilized in rapid prototyping of CUDA applications, where programmer productivity matters most, as well as in production, where robustness and absolute performance are crucial." 


`Thrust` tiene la opción de utilizarse con [OpenMP](https://www.openmp.org/), [Thread Building Blocks (TBB)](https://www.threadingbuildingblocks.org/intel-tbb-tutorial) y con CUDA-C++. Ver por ejemplo [Device Backends](https://github.com/thrust/thrust/wiki/Device-Backends) para conocer cómo cambiar entre OpenMP y CUDA-C++, lo cual se realiza en la compilación y **sin hacer cambios en el código!**. A los sistemas de software que tienen este *feature* se les llama [Heterogeneous computing](https://en.wikipedia.org/wiki/Heterogeneous_computing).

Si se instala el *CUDA toolkit* o se utiliza la imagen de docker descrita al inicio de la nota, los headers en la librería template de `Thrust` estarán disponibles para su uso.

En el siguiente ejemplo de la regla del rectángulo compuesta se utiliza:

 * [Reductions](https://docs.nvidia.com/cuda/thrust/index.html#reductions)
 
 * Los headers: 
 
    * [thrust/execution_policy](https://thrust.github.io/doc/structthrust_1_1device__execution__policy.html)
    
    * [thhrust/reduce()](https://thrust.github.io/doc/group__reductions_ga43eea9a000f912716189687306884fc7.html#ga43eea9a000f912716189687306884fc7)
 

y se hace explícito el uso de la política de ejecucion [thrust::device](https://thrust.github.io/doc/group__execution__policies_ga78249cb3aa4239b64e65aaf6e82ac2f8.html).

Referencias para el programa siguiente se encuentran en [thrust inside user written kernels](https://stackoverflow.com/questions/5510715/thrust-inside-user-written-kernels) y [cuda how to sum all elements of an array into one number within the gpu](https://stackoverflow.com/questions/42525713/cuda-how-to-sum-all-elements-of-an-array-into-one-number-within-the-gpu) de stackoverflow.

**$n=10^3$** subintervalos

In [24]:
%%file Rcf.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum_res ) {
    /*
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
        a (int): left point of interval
        n (int): number of subintervals
        data (double): array that will hold values evaluated in function
    Returns:
        sum_res (double): pointer to result
    */
    double x=0.0;
    if(threadIdx.x<=n-1){
        x=a+(threadIdx.x+1/2.0)*h_hat;
        data[threadIdx.x]=std::exp(-std::pow(x,2));
    }
    if(threadIdx.x==0){
        *sum_res = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
    }
}

int main(int argc, char *argv[]){
    double sum_res=0.0;
    double *d_data;
    double *d_sum;
    double a=0.0, b=1.0;
    double h_hat;
    int n=1e3; 
    double obj=0.7468241328124271;
    double time_spent;
    clock_t begin,end;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_sum,sizeof(double));
    h_hat=(b-a)/n;
    begin=clock();
    Rcf<<<1,n>>>(d_data, a,h_hat,n,d_sum); //1 bloque de n threads
    cudaDeviceSynchronize();
    end=clock();
    time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    cudaMemcpy(&sum_res, d_sum, sizeof(double), cudaMemcpyDeviceToHost);
    sum_res=h_hat*sum_res;
    cudaFree(d_data) ;
    cudaFree(d_sum) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,sum_res);
    printf("Error relativo de la solución: %1.15e\n", fabs(sum_res-obj)/fabs(obj));
    printf("Tiempo de cálculo en la gpu %.5f\n", time_spent);
    return 0;
}

Writing Rcf.cu


In [25]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall Rcf.cu -o Rcf.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [26]:
%%bash
./Rcf.out

Integral de 0.000000 a 1.000000 = 7.468241634690490e-01
Error relativo de la solución: 4.104931878976858e-08
Tiempo de cálculo en la gpu 0.00014


In [27]:
%%bash
source ~/.profile
nvprof --normalized-time-unit s ./Rcf.out

Integral de 0.000000 a 1.000000 = 7.468241634690490e-01
Error relativo de la solución: 4.104931878976858e-08
Tiempo de cálculo en la gpu 0.00018


==6560== NVPROF is profiling process 6560, command: ./Rcf.out
==6560== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
==6560== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the recommended nvcc version 
==6560== Profiling application: ./Rcf.out
==6560== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:    97.51  1.00e-04         1  1.00e-04  1.00e-04  1.00e-04  Rcf(double*, double, double, int, double*)
                     2.49  2.56e-06         1  2.56e-06  2.56e-06  2.56e-06  [CUDA memcpy DtoH]
      API calls:    99.53  0.246660         2  0.123330  5.87e-06  0.246654  cudaMalloc
                     0.21  5.19e-0

$n=1025$ subintervalos

In [28]:
%%file Rcf2.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum_res ) {
    /*
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
        a (int): left point of interval
        n (int): number of subintervals
        data (double): array that will hold values evaluated in function
    Returns:
        sum_res (double): pointer to result
    */
    double x=0.0;
    if(threadIdx.x<=n-1){
        x=a+(threadIdx.x+1/2.0)*h_hat;
        data[threadIdx.x]=std::exp(-std::pow(x,2));
    }
    if(threadIdx.x==0){
        *sum_res = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
    }
}

int main(int argc, char *argv[]){
    double sum_res=0.0;
    double *d_data;
    double *d_sum;
    double a=0.0, b=1.0;
    double h_hat;
    int n=1025; 
    double obj=0.7468241328124271;
    double time_spent;
    clock_t begin,end;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_sum,sizeof(double));
    h_hat=(b-a)/n;
    begin=clock();
    Rcf<<<1,n>>>(d_data, a,h_hat,n,d_sum); //1 bloque de n threads
    cudaDeviceSynchronize();
    end=clock();
    time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    cudaMemcpy(&sum_res, d_sum, sizeof(double), cudaMemcpyDeviceToHost);
    sum_res=h_hat*sum_res;
    cudaFree(d_data) ;
    cudaFree(d_sum) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,sum_res);
    printf("Error relativo de la solución: %1.15e\n", fabs(sum_res-obj)/fabs(obj));
    printf("Tiempo de cálculo en la gpu %.5f\n", time_spent);
    return 0;
}

Writing Rcf2.cu


In [29]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall Rcf2.cu -o Rcf2.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [30]:
%%bash
./Rcf2.out

Integral de 0.000000 a 1.000000 = 0.000000000000000e+00
Error relativo de la solución: 1.000000000000000e+00
Tiempo de cálculo en la gpu 0.00001


Obsérvese error relativo de $100\%$

**¿Cómo lo arreglamos?**

In [31]:
%%file Rcf3.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum_res) {
    /*
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
        a (int): left point of interval
        n (int): number of subintervals
        data (double): array that will hold values evaluated in function
    Returns:
        sum_res (double): pointer to result
    */
    double x=0.0;
    int stride=0;
    if(threadIdx.x<=n-1){
        x=a+(threadIdx.x+1/2.0)*h_hat;
        data[threadIdx.x]=std::exp(-std::pow(x,2));
    }
    if(threadIdx.x==0){
        stride=blockDim.x;
        x=a+(threadIdx.x+stride+1/2.0)*h_hat;
        data[threadIdx.x+stride]=std::exp(-std::pow(x,2));
        *sum_res = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
    }
}

int main(int argc, char *argv[]){
    double sum_res=0.0;
    double *d_data;
    double *d_sum;
    double a=0.0, b=1.0;
    double h_hat;
    int n_threads_per_block=1024; 
    int n_blocks=2;
    int n=1025;
    double obj=0.7468241328124271;
    double time_spent;
    clock_t begin,end;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_sum,sizeof(double));
    h_hat=(b-a)/n;
    begin=clock();
    Rcf<<<n_blocks,n_threads_per_block>>>(d_data, a,h_hat,n,d_sum); 
    cudaDeviceSynchronize();
    end=clock();
    time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    cudaMemcpy(&sum_res, d_sum, sizeof(double), cudaMemcpyDeviceToHost);
    sum_res=h_hat*sum_res;
    cudaFree(d_data) ;
    cudaFree(d_sum) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,sum_res);
    printf("Error relativo de la solución: %1.15e\n", fabs(sum_res-obj)/fabs(obj));
    printf("Tiempo de cálculo en la gpu %.5f\n", time_spent);
    return 0;
}

Writing Rcf3.cu


In [32]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall Rcf3.cu -o Rcf3.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [33]:
%%bash
./Rcf3.out

Integral de 0.000000 a 1.000000 = 7.468241619918411e-01
Error relativo de la solución: 3.907133247860604e-08
Tiempo de cálculo en la gpu 0.00015


Pero en la propuesta anterior lanzamos $2*1024$ (bloques $\times$ número de *threads*) $=2048$ *threads* y sólo ocupamos $1025$ threads. Entonces podemos cambiar el código anterior para aprovechar los $2048$ *threads* como sigue:

In [36]:
%%file Rcf4.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum_res) {
    /*
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
        a (int): left point of interval
        n (int): number of subintervals
        data (double): array that will hold values evaluated in function
    Returns:
        sum_res (double): pointer to result
    */
    double x=0.0;
    int stride=0;
    int i;
    stride=blockDim.x;
    for(i=threadIdx.x;i<=n-1;i+=stride){
        if(i<=n-1){
            x=a+(i+1/2.0)*h_hat;
            data[i]=std::exp(-std::pow(x,2));
        }
    }
    if(threadIdx.x==0){
        *sum_res = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
    }
}

int main(int argc, char *argv[]){
    double sum_res=0.0;
    double *d_data;
    double *d_sum;
    double a=0.0, b=1.0;
    double h_hat;
    int n_threads_per_block=1024; 
    int n_blocks=2;
    int n=n_threads_per_block*n_blocks;//número de subintervalos
    double obj=0.7468241328124271;
    double time_spent;
    clock_t begin,end;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_sum,sizeof(double));
    h_hat=(b-a)/n;
    begin=clock();
    Rcf<<<n_blocks,n_threads_per_block>>>(d_data, a,h_hat,n,d_sum); 
    cudaDeviceSynchronize();
    end=clock();
    time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    cudaMemcpy(&sum_res, d_sum, sizeof(double), cudaMemcpyDeviceToHost);
    sum_res=h_hat*sum_res;
    cudaFree(d_data) ;
    cudaFree(d_sum) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,sum_res);
    printf("Error relativo de la solución: %1.15e\n", fabs(sum_res-obj)/fabs(obj));
    printf("Tiempo de cálculo en la gpu %.5f\n", time_spent);
    return 0;
}

Writing Rcf4.cu


In [37]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall Rcf4.cu -o Rcf4.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [38]:
%%bash
./Rcf4.out

Integral de 0.000000 a 1.000000 = 7.468241401215338e-01
Error relativo de la solución: 9.786918140590463e-09
Tiempo de cálculo en la gpu 0.00024


Pero todavía podemos hacer aún mejor sin el *for*:

In [39]:
%%file Rcf5.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum_res ) {
    /*
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
        a (int): left point of interval
        n (int): number of subintervals
        data (double): array that will hold values evaluated in function
    Returns:
        sum_res (double): pointer to result
    */
    double x=0.0;
    int idx;
    idx = blockIdx.x * blockDim.x + threadIdx.x;
    if(idx<=n-1){
        x=a+(idx+1/2.0)*h_hat;
        data[idx]=std::exp(-std::pow(x,2));
    }
    if(idx==0){
        *sum_res = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
    }
}

int main(int argc, char *argv[]){
    double sum_res=0.0;
    double *d_data;
    double *d_sum;
    double a=0.0, b=1.0;
    double h_hat;
    int n_threads_per_block=1024; 
    int n_blocks=2;
    double obj=0.7468241328124271;
    int n=n_blocks*n_threads_per_block;//número de subintervalos
    double time_spent;
    clock_t begin,end;
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_sum,sizeof(double));
    h_hat=(b-a)/n;
    begin = clock();
    Rcf<<<n_blocks,n_threads_per_block>>>(d_data, a,h_hat,n,d_sum); 
    cudaDeviceSynchronize();
    end = clock();
    time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    cudaMemcpy(&sum_res, d_sum, sizeof(double), cudaMemcpyDeviceToHost);
    sum_res=h_hat*sum_res;
    cudaFree(d_data) ;
    cudaFree(d_sum) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,sum_res);
    printf("Error relativo de la solución: %1.15e\n", fabs(sum_res-obj)/fabs(obj));
    printf("Tiempo de cálculo en la gpu %.5f\n", time_spent);
    return 0;
}

Writing Rcf5.cu


In [40]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall Rcf5.cu -o Rcf5.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [41]:
%%bash
./Rcf5.out

Integral de 0.000000 a 1.000000 = 7.468241401215338e-01
Error relativo de la solución: 9.786918140590463e-09
Tiempo de cálculo en la gpu 0.00024


In [42]:
%%bash
source ~/.profile
nvprof --normalized-time-unit s ./Rcf5.out

Integral de 0.000000 a 1.000000 = 7.468241401215338e-01
Error relativo de la solución: 9.786918140590463e-09
Tiempo de cálculo en la gpu 0.00029


==6759== NVPROF is profiling process 6759, command: ./Rcf5.out
==6759== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
==6759== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the recommended nvcc version 
==6759== Profiling application: ./Rcf5.out
==6759== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:    98.70  1.95e-04         1  1.95e-04  1.95e-04  1.95e-04  Rcf(double*, double, double, int, double*)
                     1.30  2.56e-06         1  2.56e-06  2.56e-06  2.56e-06  [CUDA memcpy DtoH]
      API calls:    99.49  0.256498         2  0.128249  4.98e-06  0.256493  cudaMalloc
                     0.21  5.31e

Para una visualización sobre la construcción del índice en el kernel utilizando `blockDim.x*blockIdx.x + threadIdx.x` ver [An Even Easier Introduction to CUDA](https://devblogs.nvidia.com/even-easier-introduction-cuda/).

**¿Más nodos?**

Para este caso, incrementamos el número de bloques:

In [48]:
%%file Rcf6.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum_res ) {
    /*
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
        a (int): left point of interval
        n (int): number of subintervals
        data (double): array that will hold values evaluated in function
    Returns:
        sum_res (double): pointer to result
    */
    double x=0.0;
    int idx;
    idx = blockIdx.x * blockDim.x + threadIdx.x;
    if(idx<=n-1){
        x=a+(idx+1/2.0)*h_hat;
        data[idx]=std::exp(-std::pow(x,2));
    }
    if(idx==0){
        *sum_res = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
    }
}

int main(int argc, char *argv[]){
    double sum_res=0.0;
    double *d_data;
    double *d_sum;
    double a=0.0, b=1.0;
    double h_hat;
    int n_threads_per_block=1024; 
    int n_blocks=0; //inicializamos en 0
    double obj=0.7468241328124271;
    int n=0; //inicializamos en 0
    double time_spent;
    clock_t begin,end;
    cudaDeviceProp properties;
    cudaGetDeviceProperties(&properties, 0);
    n_blocks = 256 * properties.multiProcessorCount; //elegimos un múltiplo del número de SM's
                                                    //y además que sea menor a 10^4 (para tener 10^6 aprox subintervalos)
    n = n_blocks*n_threads_per_block;//número de subintervalos
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_sum,sizeof(double));
    h_hat=(b-a)/n;
    begin = clock();
    Rcf<<<n_blocks,n_threads_per_block>>>(d_data, a,h_hat,n,d_sum); 
    cudaDeviceSynchronize();
    end = clock();
    time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    cudaMemcpy(&sum_res, d_sum, sizeof(double), cudaMemcpyDeviceToHost);
    sum_res=h_hat*sum_res;
    cudaFree(d_data) ;
    cudaFree(d_sum) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,sum_res);
    printf("Error relativo de la solución: %1.15e\n", fabs(sum_res-obj)/fabs(obj));
    printf("Tiempo de cálculo en la gpu %.5f\n", time_spent);
    return 0;
}

Writing Rcf6.cu


In [49]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall Rcf6.cu -o Rcf6.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [50]:
%%bash
./Rcf6.out

Integral de 0.000000 a 1.000000 = 7.468241328124654e-01
Error relativo de la solución: 5.128743524305478e-14
Tiempo de cálculo en la gpu 0.42348


In [51]:
%%bash
source ~/.profile
nvprof --normalized-time-unit s ./Rcf6.out

Integral de 0.000000 a 1.000000 = 7.468241328124654e-01
Error relativo de la solución: 5.128743524305478e-14
Tiempo de cálculo en la gpu 0.43533


==6837== NVPROF is profiling process 6837, command: ./Rcf6.out
==6837== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
==6837== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the recommended nvcc version 
==6837== Profiling application: ./Rcf6.out
==6837== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:   100.00  0.434770         1  0.434770  0.434770  0.434770  Rcf(double*, double, double, int, double*)
                     0.00  2.59e-06         1  2.59e-06  2.59e-06  2.59e-06  [CUDA memcpy DtoH]
      API calls:    60.64  0.434787         1  0.434787  0.434787  0.434787  cudaDeviceSynchronize
                    3

$n=16777216 \approx 10^7$ subintervalos?

In [58]:
%%file Rcf7.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, int n, double *sum_res ) {
    /*
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
        a (int): left point of interval
        n (int): number of subintervals
        data (double): array that will hold values evaluated in function
    Returns:
        sum_res (double): pointer to result
    */
    double x=0.0;
    int idx;
    idx = blockIdx.x * blockDim.x + threadIdx.x;
    if(idx<=n-1){
        x=a+(idx+1/2.0)*h_hat;
        data[idx]=std::exp(-std::pow(x,2));
    }
    if(idx==0){
        *sum_res = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
    }
}

int main(int argc, char *argv[]){
    double sum_res=0.0;
    double *d_data;
    double *d_sum;
    double a=0.0, b=1.0;
    double h_hat;
    int n_threads_per_block=1024; 
    int n_blocks=0; //inicializamos en 0
    double obj=0.7468241328124271;
    int n=0; //inicializamos en 0
    double time_spent;
    clock_t begin,end;
    cudaDeviceProp properties;
    cudaGetDeviceProperties(&properties, 0);
    n_blocks = 4096 * properties.multiProcessorCount; //elegimos un múltiplo del número de SM's
                                                    //y además que sea menor a 10^8 (para tener 10^7 aprox subintervalos)
    n = n_blocks*n_threads_per_block;//número de subintervalos
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_sum,sizeof(double));
    h_hat=(b-a)/n;
    begin = clock();
    Rcf<<<n_blocks,n_threads_per_block>>>(d_data, a,h_hat,n,d_sum); 
    cudaDeviceSynchronize();
    end = clock();
    time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    cudaMemcpy(&sum_res, d_sum, sizeof(double), cudaMemcpyDeviceToHost);
    sum_res=h_hat*sum_res;
    cudaFree(d_data) ;
    cudaFree(d_sum) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,sum_res);
    printf("Error relativo de la solución: %1.15e\n", fabs(sum_res-obj)/fabs(obj));
    printf("Tiempo de cálculo en la gpu %.5f\n", time_spent);
    return 0;
}

Writing Rcf7.cu


In [59]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall Rcf7.cu -o Rcf7.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [60]:
%%bash
./Rcf7.out

Integral de 0.000000 a 1.000000 = 7.468241328130133e-01
Error relativo de la solución: 7.849207480676210e-13
Tiempo de cálculo en la gpu 5.47597


In [62]:
%%bash
source ~/.profile
nvprof --normalized-time-unit s ./Rcf7.out

Integral de 0.000000 a 1.000000 = 7.468241328130133e-01
Error relativo de la solución: 7.849207480676210e-13
Tiempo de cálculo en la gpu 5.33121


==6952== NVPROF is profiling process 6952, command: ./Rcf7.out
==6952== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
==6952== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the recommended nvcc version 
==6952== Profiling application: ./Rcf7.out
==6952== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:   100.00  5.327689         1  5.327689  5.327689  5.327689  Rcf(double*, double, double, int, double*)
                     0.00  2.18e-06         1  2.18e-06  2.18e-06  2.18e-06  [CUDA memcpy DtoH]
      API calls:    94.95  5.327721         1  5.327721  5.327721  5.327721  cudaDeviceSynchronize
                     

$n=117440512 \approx 10^8$ subintervalos?

In [69]:
%%file Rcf8.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, long int n, double *sum_res ) {
    /*
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
        a (int): left point of interval
        n (int): number of subintervals
        data (double): array that will hold values evaluated in function
    Returns:
        sum_res (double): pointer to result
    */
    double x=0.0;
    int idx;
    idx = blockIdx.x * blockDim.x + threadIdx.x;
    if(idx<=n-1){
        x=a+(idx+1/2.0)*h_hat;
        data[idx]=std::exp(-std::pow(x,2));
    }
    //esta alternativa no funciona... si el número de bloques rebasa el límite:
    //int num_threads=gridDim.x * blockDim.x;
    //int stride = num_threads;
    //for(i=idx; i<=n-1; i+=stride){
    //    if(idx<=n-1){
    //        x=a+(idx+1/2.0)*h_hat;
    //        data[idx]=std::exp(-std::pow(x,2));
    //    }
    //}
    
    if(idx==0){
        *sum_res = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
    }
}

int main(int argc, char *argv[]){
    double sum_res=0.0;
    double *d_data;
    double *d_sum;
    double a=0.0, b=1.0;
    double h_hat;
    int n_threads_per_block=1024; 
    long int n_blocks=0; //inicializamos en 0
    double obj=0.7468241328124271;
    long int n=0; //inicializamos en 0
    double time_spent;
    clock_t begin,end;
    cudaDeviceProp properties;    
    cudaGetDeviceProperties(&properties, 0);
    n_blocks = 8192 * properties.multiProcessorCount; //elegimos un múltiplo del número de SM's
                                                    //y además que sea menor a 10^9 (para tener 10^8 aprox subintervalos)
    n = n_blocks*n_threads_per_block;//número de subintervalos
    dim3 dimGrid(n_blocks,1,1);
    dim3 dimBlock(n_threads_per_block,1,1);
    cudaMalloc((void **)&d_data,sizeof(double)*n);
    cudaMalloc((void**)&d_sum,sizeof(double));
    h_hat=(b-a)/n;
    begin = clock();
    Rcf<<<dimGrid,dimBlock>>>(d_data, a,h_hat,n,d_sum); 
    cudaDeviceSynchronize();
    end = clock();
    time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    cudaMemcpy(&sum_res, d_sum, sizeof(double), cudaMemcpyDeviceToHost);
    sum_res=h_hat*sum_res;
    cudaFree(d_data) ;
    cudaFree(d_sum) ;
    printf("Integral de %f a %f = %1.15e\n", a,b,sum_res);
    printf("Error relativo de la solución: %1.15e\n", fabs(sum_res-obj)/fabs(obj));
    printf("Tiempo de cálculo en la gpu %.5f\n", time_spent);
    return 0;
}

Writing Rcf8.cu


In [70]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall Rcf8.cu -o Rcf8.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [71]:
%%bash
./Rcf8.out

Integral de 0.000000 a 1.000000 = 7.468241328130135e-01
Error relativo de la solución: 7.852180665327982e-13
Tiempo de cálculo en la gpu 10.78316


In [72]:
%%bash
source ~/.profile
nvprof --normalized-time-unit s ./Rcf8.out

Integral de 0.000000 a 1.000000 = 7.468241328130135e-01
Error relativo de la solución: 7.852180665327982e-13
Tiempo de cálculo en la gpu 10.81419


==7043== NVPROF is profiling process 7043, command: ./Rcf8.out
==7043== Warning: Auto boost enabled on device 0. Profiling results may be inconsistent.
==7043== Warning: Profiling results might be incorrect with current version of nvcc compiler used to compile cuda app. Compile with nvcc compiler 9.0 or later version to get correct profiling results. Ignore this warning if code is already compiled with the recommended nvcc version 
==7043== Profiling application: ./Rcf8.out
==7043== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
                        %         s                   s         s         s
 GPU activities:   100.00  10.80508         1  10.80508  10.80508  10.80508  Rcf(double*, double, double, long, double*)
                     0.00  2.34e-06         1  2.34e-06  2.34e-06  2.34e-06  [CUDA memcpy DtoH]
      API calls:    97.05  10.80505         1  10.80505  10.80505  10.80505  cudaDeviceSynchronize
                    

**Obs:** en la programación con CUDA-C es importante checar posibles errores de alojamiento de memoria. Una forma es con los tipos [cudaError_t](https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html#group__CUDART__TYPES_1gf599e5b8b829ce7db0f5216928f6ecb6) y `cudaSuccess` . Ver [liga](https://stackoverflow.com/questions/58902166/why-do-i-have-insufficient-buffer-space-when-i-put-allocation-code-in-a-functi):

In [77]:
%%file Rcf9.cu
#include<stdio.h>
#include <thrust/reduce.h>
#include <thrust/execution_policy.h>

__global__ void Rcf(double *data, double a, double h_hat, long int n, double *sum_res ) {
    /*
    Compute numerical approximation using rectangle or mid-point method in 
    an interval.
    Nodes are generated via formula: x_i = a+(i+1/2)h_hat for i=0,1,...,n-1 and h_hat=(b-a)/n
    Args:
        a (int): left point of interval
        n (int): number of subintervals
        data (double): array that will hold values evaluated in function
    Returns:
        sum_res (double): pointer to result
    */
    double x=0.0;
    int idx;
    int num_threads=gridDim.x * blockDim.x;
    int stride = num_threads;
    int i;
    idx = blockIdx.x * blockDim.x + threadIdx.x;
    for(i=idx; i<=n-1; i+=stride){
        if(idx<=n-1){
            x=a+(idx+1/2.0)*h_hat;
            data[idx]=std::exp(-std::pow(x,2));
        }
}
    
    if(idx==0){
        *sum_res = thrust::reduce(thrust::device, data , data + n, (double)0, thrust::plus<double>());
    }
}

cudaError_t check_error(cudaError_t result) {
    if (result != cudaSuccess) {
        fprintf(stderr, "Error: %s\n", cudaGetErrorString(result));
    }
    return result;
}

int main(int argc, char *argv[]){
    double sum_res=0.0;
    double *d_data;
    double *d_sum;
    double a=0.0, b=1.0;
    double h_hat;
    int n_threads_per_block=1024; 
    long int n_blocks=0; //inicializamos en 0
    double obj=0.7468241328124271;
    long int n=0; //inicializamos en 0
    double time_spent;
    clock_t begin,end;
    cudaDeviceProp properties;    
    cudaGetDeviceProperties(&properties, 0);
    n_blocks = 32768 * properties.multiProcessorCount; //elegimos un múltiplo del número de SM's
                                                    //y además que sea menor a 10^9 (para tener 10^8 aprox subintervalos)
    n = n_blocks*n_threads_per_block;//número de subintervalos
    dim3 dimGrid(n_blocks,1,1);
    dim3 dimBlock(n_threads_per_block,1,1);
    check_error(cudaMalloc((void **)&d_data,sizeof(double)*n));
    check_error(cudaMalloc((void**)&d_sum,sizeof(double)));
    h_hat=(b-a)/n;
    begin = clock();
    Rcf<<<dimGrid,dimBlock>>>(d_data, a,h_hat,n,d_sum); 
    cudaDeviceSynchronize();
    end = clock();
    time_spent = (double)(end - begin) / CLOCKS_PER_SEC;
    check_error(cudaMemcpy(&sum_res, d_sum, sizeof(double), cudaMemcpyDeviceToHost));
    sum_res=h_hat*sum_res;
    cudaFree(d_data);
    cudaFree(d_sum);
    printf("Integral de %f a %f = %1.15e\n", a,b,sum_res);
    printf("Error relativo de la solución: %1.15e\n", fabs(sum_res-obj)/fabs(obj));
    printf("Tiempo de cálculo en la gpu %.5f\n", time_spent);
    return 0;
}

Writing Rcf9.cu


In [78]:
%%bash
source ~/.profile
nvcc -gencode arch=compute_37,code=sm_37 --compiler-options -Wall Rcf9.cu -o Rcf9.out

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).


In [79]:
%%bash
./Rcf9.out

Integral de 0.000000 a 1.000000 = 7.468241328124106e-01
Error relativo de la solución: 2.215022565569613e-14
Tiempo de cálculo en la gpu 42.72796


## [CuPy](https://github.com/cupy/cupy)

## [Gputools](https://github.com/nullsatz/gputools) 

Ver [gputools: cran](https://rdrr.io/cran/gputools/)

## Referencias de interés

Para más sobre *Unified Memory* revisar:

* [Even easier introduction to cuda](https://devblogs.nvidia.com/even-easier-introduction-cuda/)

* [Unified memory cuda beginners](https://devblogs.nvidia.com/unified-memory-cuda-beginners/)

Es importante el manejo de errores por ejemplo en el alojamiento de memoria en la GPU. En este caso es útil revisar:

* [How to Query Device Properties and Handle Errors in CUDA C/C++](https://devblogs.nvidia.com/how-query-device-properties-and-handle-errors-cuda-cc/)

En las siguientes preguntas encontramos a personas desarrolladoras de CUDA que las resuelven y resultan muy útiles para continuar con el aprendizaje de CUDA C. Por ejemplo: 

* [Parallel reduction over one axis](https://stackoverflow.com/questions/51526082/cuda-parallel-reduction-over-one-axis)

Otros sistemas de software para el [Heterogeneous computing](https://en.wikipedia.org/wiki/Heterogeneous_computing) son:

* [OpenCl](https://en.wikipedia.org/wiki/OpenCL). Ver [NVIDIA OpenCL SDK Code Samples](https://developer.nvidia.com/opencl) para ejemplos con NVIDIA GPU's.

* [Rth-org/Rth](https://github.com/Rth-org/Rth) y más reciente [matloff/Rth](https://github.com/matloff/Rth). Ver también [rdrr.io matloff/Rth](https://rdrr.io/github/matloff/Rth/f/README.md).

Es posible escribir [kernels](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/blob/master/temas/II.computo_paralelo/2.3.CUDA_C.ipynb) con CuPy. Ver por ejemplo: [User-Defined Kernels](https://docs-cupy.chainer.org/en/stable/tutorial/kernel.html).

* [CuPy – NumPy-like API accelerated with CUDA](https://docs-cupy.chainer.org/en/stable/index.html)

* [CuPy : NumPy-like API accelerated with CUDA github](https://github.com/cupy/cupy)

Otro paquete para uso de Python+GPU para cómputo matricial es:

* [PyCUDA](https://github.com/inducer/pycuda/) y ver [PyCUDA en el repo de la clase](https://github.com/ITAM-DS/analisis-numerico-computo-cientifico/tree/master/Python/PyCUDA) para más información.

Un paquete para uso de pandas+GPU:

* [Rapids](https://github.com/rapidsai), [cudf](https://github.com/rapidsai/cudf)

Ver [optional-libraries](https://docs-cupy.chainer.org/en/stable/install.html#optional-libraries) para librerías que pueden ser utilizadas con CuPy.


```{admonition} Ejercicios
:class: tip

1.Resuelve los ejercicios y preguntas de la nota.
```


**Preguntas de comprehensión:**

1)¿Qué factores han determinado un mejor *performance* de una GPU vs una CPU? (contrasta los diseños de una CPU vs una GPU).

2)¿Dentro de qué modelo de arquitectura de máquinas se ubica a la GPU dentro de la taxonomía de Flynn? (tip: tal modelo se le puede comparar con el modelo **Single Program Multiple Data (SPMD)**)

3)¿Qué significan las siglas CUDA y detalla qué es CUDA?.

4)¿Qué es y en qué consiste CUDA C?
    
5)¿Qué es un *kernel*?

6)¿Qué pieza de CUDA se encarga de asignar los bloques de *cuda-threads* a las SM’s?

7)¿Qué características (recursos compartidos, dimensiones, forma de agendar la ejecución en *threads*) tienen los bloques que se asignan a una SM al lanzarse y ejecutarse un *kernel*?

8)¿Qué es un *warp*?

9)Menciona los tipos de memorias que existen en las GPU’s.

10)Supón que tienes una tarjeta GT200 cuyas características son:

* Máximo número de *threads* que soporta una SM en un mismo instante en el tiempo: 1024
* Máximo número de *threads* en un bloque: 512
* Máximo número de bloques por SM: 8
* Número de SM’s que tiene esta GPU: 30

Responde:

a)¿Cuál es la máxima cantidad de *threads* que puede soportar esta GPU en un mismo instante en el tiempo?

b)¿Cuál es la máxima cantidad de *warps* por SM que puede soportar esta GPU en un mismo instante en el tiempo?

c)¿Cuáles configuraciones de bloques y *threads* siguientes aprovechan la máxima cantidad de *warps* en una SM de esta GPU para un mismo instante en el tiempo?
    
1.Una configuración del tipo: bloques de 64 *threads* y 16 bloques.

2.Una configuración del tipo: bloques de 1024 *threads* y 1 bloque.

3.Una configuración del tipo: bloques de 256 *threads* y 4 bloques.

4.Una configuración del tipo: bloques de 512 *threads* y 8 bloques.

\*Debes considerar las restricciones/características de la GPU dadas para responder pues algunas configuraciones infringen las mismas. No estamos considerando *registers* o *shared memory*.


**Referencias:**

1. N. Matloff, Parallel Computing for Data Science. With Examples in R, C++ and CUDA, 2014.

2. D. B. Kirk, W. W. Hwu, Programming Massively Parallel Processors: A Hands-on Approach, Morgan Kaufmann, 2010.

3. NVIDIA,CUDA Programming Guide, NVIDIA Corporation, 2007.

4. B. W. Kernighan, D. M. Ritchie, The C Programming Language, Prentice Hall Software Series, 1988

5. [C/extensiones_a_C/CUDA/](https://github.com/palmoreck/programming-languages/tree/master/C/extensiones_a_C/CUDA)